# DenoiSeg Example: Fly Wing
This is an example notebook which illustrates how DenoiSeg should be trained. In this notebook we use a membrane labeled developing Fly Wing dataset from our collaborators. We already split the data into train and test images. From the train images we then extracted 1428 training and 252 validation patches of size 128x128. The test set contains 50 images of size 512x512.

In [3]:
!pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.5 MB/s eta 0:00:0000:0100:05
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 6.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.9/22.9 MB 3.3 MB/s eta 0:00:0000:0100:01
  Using cached numpy-1.24.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.3 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 6.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━

In [2]:
# Here we are just importing some libraries which are needed to run this notebook.
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage

from denoiseg.models import DenoiSeg, DenoiSegConfig
from denoiseg.utils.misc_utils import combine_train_test_data, shuffle_train_data, augment_data
from denoiseg.utils.seg_utils import *
from denoiseg.utils.compute_precision_threshold import measure_precision
from denoiseg.utils.denoiseg_data_preprocessing import generate_patches_from_list

from csbdeep.utils import plot_history

import urllib
import os
import zipfile

ModuleNotFoundError: No module named 'tensorflow'

## Downloading and  Data Loading
We created three versions of this dataset by adding Gaussian noise with zero mean and standard deviations 10 and 20. The dataset are marked with the suffixes n0, n10 and n20 accordingly.

In the next cell you can choose which `noise_level` you would like to investigate.

In [ ]:
# Choose the noise level you would like to look at:
# Values: 'n0', 'n10', 'n20'
# noise_level = 'n20'

In [ ]:
# create a folder for our data
if not os.path.isdir('./data'):
    os.mkdir('data')
    
if noise_level == 'n0':
    link = 'https://zenodo.org/record/5156991/files/Flywing_n0.zip?download=1'
elif noise_level == 'n10':
    link = 'https://zenodo.org/record/5156993/files/Flywing_n10.zip?download=1'
elif noise_level == 'n20':
    link = 'https://zenodo.org/record/5156995/files/Flywing_n20.zip?download=1'
else:
    print('This noise level does not exist for this dataset.')

# check if data has been downloaded already
zipPath="data/Flywing_{}.zip".format(noise_level)
if not os.path.exists(zipPath):
    #download and unzip data
    data = urllib.request.urlretrieve(link, zipPath)
    with zipfile.ZipFile(zipPath, 'r') as zip_ref:
        zip_ref.extractall("data")

In [ ]:
# Loading of the training images
trainval_data =  np.load('/home/sasha/WSLProjects/denoising/DenoiSeg/scripts/reproducibility/dataprep/data/train/train_data.npz')

train_images = trainval_data['X_train'].astype(np.float32)
train_masks = trainval_data['Y_train']
val_images = trainval_data['X_val'].astype(np.float32)
val_masks = trainval_data['Y_val']

In [ ]:
print("Shape of train images: {}".format(train_images.shape))
print("Shape of train masks: {}".format(train_masks.shape))
print("Shape of validation images: {}".format(val_images.shape))
print("Shape of validation: {}".format(val_masks.shape))

## Small Amounts of Annotated Training Data
With DenoiSeg we present a solution to train deep neural networks if only few annotated ground truth segmentations are available. We simulate such a scenario by zeroing out all but a fraction of the available training data. In the next cell you can specify the percentage of training images for which ground truth annotations are available.

In [ ]:
# Set the number of annotated training images.
# Values: 0.0 (no annotated images) to total number of training images (all images have annotations)
number_of_annotated_training_images = 5
assert number_of_annotated_training_images >= 0.0 and number_of_annotated_training_images <=train_images.shape[0]

In [ ]:
# Seed to shuffle training data (annotated GT and raw image pairs).
seed = 1 

# First we shuffle the training images to remove any bias.
X_shuffled, Y_shuffled = shuffle_train_data(train_images, train_masks, random_seed=seed)

# Here we convert the number of annotated images to be used for training as percentage of available training data.
percentage_of_annotated_training_images = float((number_of_annotated_training_images/train_images.shape[0]) * 100.0)
assert percentage_of_annotated_training_images >= 0.0 and percentage_of_annotated_training_images <= 100.0

# Here we zero out all training images which are not part of the 
# selected percentage.
X_frac, Y_frac = zero_out_train_data(train_images, train_masks, fraction=percentage_of_annotated_training_images)

# Here we generate patches from images and apply augmentation
X_final, Y_final = generate_patches_from_list([X_frac], [Y_frac], augment=True, shuffle=False, shape=(128, 128))
X_val_final, Y_val_final = generate_patches_from_list([val_images], [val_masks], augment=False, shape=(128, 128))

X_final = X_final[... ,np.newaxis]
Y_final = convert_to_oneHot(Y_final, n_classes=3)

X_val_final = X_val_final[... ,np.newaxis]
Y_val_final = convert_to_oneHot(Y_val_final, n_classes=3)

print("Shape of train images after augmentation: {}".format(X_final.shape))
print("Shape of train masks after augmentation: {}".format(Y_final.shape))

Next we look at a single sample. In the first column we show the input image, in the second column the background segmentation, in the third column the foreground segmentation and in the last column the border segmentation.

With the parameter `sample` you can choose different training patches. You will notice that not all of them have a segmentation ground truth.

In [ ]:
sample = 0
plt.figure(figsize=(20,5))
plt.subplot(1,4,1)
plt.imshow(X_final[sample,...,0])
plt.axis('off')
plt.title('Raw validation image')
plt.subplot(1,4,2)
plt.imshow(Y_final[sample,...,0], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded background')
plt.subplot(1,4,3)
plt.imshow(Y_final[sample,...,1], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded foreground')
plt.subplot(1,4,4)
plt.imshow(Y_final[sample,...,2], vmin=0, vmax=1, interpolation='nearest')
plt.axis('off')
plt.title('1-hot encoded border')

## Configure network parameters

In [ ]:
train_batch_size = 128
train_steps_per_epoch = min(400, max(int(X_final.shape[0]/train_batch_size), 10))

In the next cell, you can choose how much relative importance (weight) to assign to denoising
and segmentation tasks by choosing appropriate value for denoiseg_alpha (between 0 and 1; with 0 being
only segmentation and 1 being only denoising. Here we choose denoiseg_alpha = 0.5)

In [ ]:
conf = DenoiSegConfig(X_final, unet_kern_size=3, n_channel_in=1, n_channel_out=4, relative_weights = [1.0,1.0,5.0],
                      train_steps_per_epoch=train_steps_per_epoch, train_epochs=10,
                      batch_norm=True, train_batch_size=train_batch_size, unet_n_first=32,
                      unet_n_depth=4, denoiseg_alpha=0.5, train_tensorboard=False)

vars(conf)

In [ ]:
model_name = 'SBER'
basedir = 'models'
model = DenoiSeg(conf, model_name, basedir)

In [ ]:
history = model.train(X_final, Y_final, (X_val_final, Y_val_final))

In [ ]:
history.history.keys()

In [ ]:
plot_history(history, ['loss', 'val_loss'])

## Computing Threshold Value
The network predicts 4 output channels:
1. The denoised input.
2. The foreground likelihoods.
3. The background likelihoods.
4. The border likelihoods.

We will threshold the foreground prediction image to obtain object segmentations. The optimal threshold is determined on the validation data. Additionally we can optimize the threshold for a given measure. In this case we choose the Average Precision (AP) measure.

In [ ]:
threshold, val_score = model.optimize_thresholds(val_images.astype(np.float32), val_masks, measure=measure_precision(), axes='YX')

print("The higest score of {} is achieved with threshold = {}.".format(np.round(val_score, 3), threshold))

## Test Data
Finally we load the test data and run the prediction.

In [ ]:
test_data =  np.load('/home/sasha/WSLProjects/denoising/DenoiSeg/scripts/reproducibility/dataprep/data/test/test_data.npz', allow_pickle=True)
test_images = test_data['X_test']
test_masks = test_data['Y_test']

In [ ]:
predicted_denoised, predicted_images, precision_result, _ = model.predict_denoised_label_masks(resval_x,
                                                                                               resval_y,
                                                                                               axes='YX',
                                                                                               threshold=threshold,
                                                                                               measure=measure_precision(),
                                                                                               n_tiles=4)

print("Average precision over all test images with threshold = {} is {}.".format(threshold, np.round(precision_result, 3)))

### Visualize the results

In [ ]:
sl = 1
fig = plt.figure()
plt.figure(figsize=(20,10))
plt.subplot(1, 4, 1)
plt.imshow(test_images[sl])
plt.title("Raw image")

plt.subplot(1, 4, 2)
plt.imshow(predicted_denoised[sl])
plt.title("Predicted denoised image")

plt.subplot(1, 4, 3)
plt.imshow(predicted_images[sl])
plt.title("Predicted foreground segmentation")

plt.subplot(1, 4, 4)
plt.imshow(test_masks[sl])
plt.title("Ground truth segmentation")

plt.show()

### Export your model for Fiji

In [ ]:
model.export_TF(name='DenoiSeg - SBER Example', 
                description='This is the 2D DenoiSeg example trained on SBER dataset', 
                authors=["SBER"],
                test_img=X_val[0,...,0], axes='YX',
                patch_shape=(128, 128))